In [2]:
!pip install ultralytics


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
import shutil

def convert_annotations(json_file, img_dir, output_label_dir, output_image_dir):
    with open(json_file) as f:
        annotations = json.load(f)

    os.makedirs(output_label_dir, exist_ok=True)
    os.makedirs(output_image_dir, exist_ok=True)

    for annotation in annotations:
        if len(annotation) < 3 or not annotation[2]:
            print(f"Skipping annotation due to missing or empty bounding boxes: {annotation}")
            continue

        img_path = annotation[0]
        label = annotation[1]
        bboxes = annotation[2]

        if label == 'person':
            label_id = 1
        elif label == 'pet':
            label_id = 0
        else:
            print(f"Skipping unknown label: {label}")
            continue

        # Calculate YOLO format bbox
        img_name = os.path.basename(img_path)
        txt_name = img_name.replace(".jpg", ".txt")
        output_label_path = os.path.join(output_label_dir, txt_name)
        output_image_path = os.path.join(output_image_dir, img_name)

        # Copy image to the new directory
        shutil.copyfile(os.path.join(img_dir, img_name), output_image_path)

        height, width = 500, 500  # Assuming a fixed size, adjust based on your data
        yolo_bboxes = []
        if isinstance(bboxes[0], list):  # Check if the bboxes are in a list of lists
            for box in bboxes:
                if len(box) != 4:
                    print(f"Skipping bbox due to incorrect number of fields: {box}")
                    continue

                x_center = (box[0] + box[2]) / 2.0 / width
                y_center = (box[1] + box[3]) / 2.0 / height
                box_width = (box[2] - box[0]) / width
                box_height = (box[3] - box[1]) / height
                yolo_bboxes.append(f"{label_id} {x_center} {y_center} {box_width} {box_height}")
        else:  # If bboxes is a single list of 4 elements
            if len(bboxes) == 4:
                x_center = (bboxes[0] + bboxes[2]) / 2.0 / width
                y_center = (bboxes[1] + bboxes[3]) / 2.0 / height
                box_width = (bboxes[2] - bboxes[0]) / width
                box_height = (bboxes[3] - bboxes[1]) / height
                yolo_bboxes.append(f"{label_id} {x_center} {y_center} {box_width} {box_height}")
            else:
                print(f"Skipping bbox due to incorrect number of fields: {bboxes}")

        with open(output_label_path, "w") as outfile:
            outfile.write("\n".join(yolo_bboxes))

# Example usage with adjusted paths:
convert_annotations('/content/drive/MyDrive/annotations3_train.json', '/content/drive/MyDrive/flickr30k_images/flickr30k_images', '/content/drive/MyDrive/dataset/labels/train', '/content/drive/MyDrive/dataset/images/train')
convert_annotations('/content/drive/MyDrive/annotations3_val.json', '/content/drive/MyDrive/flickr30k_images/flickr30k_images', '/content/drive/MyDrive/dataset/labels/val', '/content/drive/MyDrive/dataset/images/val')


In [ ]:
df= '/content/drive/MyDrive/dataset/labels/val'
len(os.listdir(df))

200

# Train model without augmentations

In [18]:
import os
import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Define base path
BASE_PATH ='/content/drive/MyDrive/dataset'

# Create dataset YAML configuration
dataset_yaml = {
    'train': '/content/drive/MyDrive/dataset/images/train',
    'val': '/content/drive/MyDrive/dataset/images/val',
    'nc': 2,
    'names': ["pet", "person"],  # class names
    'augment': 0 #disalbe augmantation
}

# Write to a YAML file
yaml_file_path = os.path.join(BASE_PATH, 'mydataset.yaml')
with open(yaml_file_path, 'w') as file:
    yaml.dump(dataset_yaml, file, default_flow_style=False)

print(f'YAML file created: {yaml_file_path}')




YAML file created: /content/drive/MyDrive/dataset/mydataset.yaml


In [19]:

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Use the appropriate model version

# Train the model with augmentations
results = model.train(data=yaml_file_path, epochs=60, lr0=0.0001, lrf=0.01, batch=64,
                      weight_decay=1e-3, patience=15, cos_lr=True, device='cuda:0',
                      optimizer='RAdam', augment=False)

# Save the model
model_path = os.path.join( '/content/drive/MyDrive/dataset/modelNoAugment.pt')
model.save(model_path)
print(f'Model saved at: {model_path}')


Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/dataset/mydataset.yaml, epochs=60, time=None, patience=15, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=RAdam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

train: Scanning /content/drive/MyDrive/dataset/labels/train.cache... 977 images, 0 backgrounds, 0 corrupt: 100%|██████████| 977/977 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/dataset/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: RAdam(lr=0.0001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      58/60      10.1G      1.447       2.06      1.415         73        640: 100%|██████████| 16/16 [00:05<00:00,  2.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.77it/s]

                   all        200        597      0.731      0.525      0.565      0.326
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 43, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



58 epochs completed in 0.132 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.46it/s]


                   all        200        597      0.717      0.528      0.578       0.34
                   pet        100        148      0.754      0.662      0.714      0.433
                person        100        449      0.681      0.394      0.442      0.246
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train
Model saved at: /content/drive/MyDrive/dataset/modelNoAugment.pt


# validation

In [16]:
# Validate the model
metrics = model.val()
print(metrics.box.map)

Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/dataset/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.16s/it]


                   all        200        597      0.707      0.533      0.577      0.336
                   pet        100        148       0.75      0.662      0.717      0.432
                person        100        449      0.664      0.403      0.438       0.24
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 7.9ms postprocess per image
Results saved to runs/detect/train2
0.3362035541841923


# Train with augmentations

In [20]:
import os
import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Define base path
BASE_PATH ='/content/drive/MyDrive/dataset'

# Create dataset YAML configuration
dataset_yaml = {
    'train': '/content/drive/MyDrive/dataset/images/train',
    'val': '/content/drive/MyDrive/dataset/images/val',
    'nc': 2,
    'names': ["pet", "person"],  # class names
    'augmentation:': 1  # enable augmantation
}

# Write to a YAML file
yaml_file_path = os.path.join(BASE_PATH, 'mydataset2.yaml')
with open(yaml_file_path, 'w') as file:
    yaml.dump(dataset_yaml, file, default_flow_style=False)

print(f'YAML file created: {yaml_file_path}')




YAML file created: /content/drive/MyDrive/dataset/mydataset2.yaml


In [21]:

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Use the appropriate model version

# Train the model with augmentations
results = model.train(data=yaml_file_path, epochs=60, lr0=0.0001, lrf=0.01, batch=64,
                      weight_decay=1e-3, patience=15, cos_lr=True, device='cuda:0',
                      optimizer='RAdam', augment=True, hsv_h=0.02, hsv_s=0.22,
                      hsv_v=0.22, degrees=2.0, shear=0.15, perspective=0.0001,
                      flipud=0.03, fliplr=0.03, mosaic=0.1, mixup=0.1, erasing=0.02,
                      crop_fraction=0.06)

# Save the model
model_path = os.path.join( '/content/drive/MyDrive/dataset/best_model.pt')
model.save(model_path)
print(f'Model saved at: {model_path}')


Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/dataset/mydataset2.yaml, epochs=60, time=None, patience=15, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=RAdam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

train: Scanning /content/drive/MyDrive/dataset/labels/train.cache... 977 images, 0 backgrounds, 0 corrupt: 100%|██████████| 977/977 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/dataset/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: RAdam(lr=0.0001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.001), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      60/60        10G      1.341      1.712      1.349         58        640: 100%|██████████| 16/16 [00:05<00:00,  2.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.90it/s]

                   all        200        597      0.751       0.54      0.615      0.356



60 epochs completed in 0.153 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


                   all        200        597      0.656      0.525      0.582      0.344
                   pet        100        148       0.67      0.662      0.707      0.429
                person        100        449      0.642      0.388      0.457      0.258
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train2
Model saved at: /content/drive/MyDrive/dataset/best_model.pt


# validation

In [22]:
# Validate the model
metrics = model.val()
print(metrics.box.map)

Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/dataset/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.30s/it]


                   all        200        597      0.638      0.543      0.577       0.35
                   pet        100        148      0.647      0.682      0.705      0.445
                person        100        449      0.629      0.404      0.449      0.256
Speed: 0.2ms preprocess, 6.0ms inference, 0.0ms loss, 8.4ms postprocess per image
Results saved to runs/detect/train22
0.3504713641362827


In [17]:
import shutil

# Replace 'path_to_directory' with the path of the directory you want to delete
shutil.rmtree('/content/runs')
